In [9]:
import numpy as np
from gurobipy import *
import random

In [49]:
def connected(vertex,adjacency,n,visited):
    visited[vertex] = 1
    row = adjacency[vertex]
    
    for node in range(n):
        if row[node] > 0 and visited[node] < 1:
            visited = connected(node,adjacency,n,visited)
    
    return visited

def rand_DAG(N,p):
    go = True
    while go:
        dag = np.zeros([N,N])
        for i in np.arange(N):
            for j in np.arange(i+1,N):
                if (random.random() < p):
                    dag[i][j] = 1
        visited = np.zeros(N)
        visited = connected(0,dag,N,visited)
        if(visited[N-1] == 1):
            go = False
            for i in range(N-1):
                if visited[i] == 0:
                    dag[i] = np.zeros(N)
    for i in range(N):
        visited = np.zeros(N)
        visited = connected(i,dag,N,visited)
        if(visited[N-1] == 0):
            dag[:,i] = np.zeros(N)
    return dag

def solve_TOP(adjacency,n,k,prizes,numPaths,s):
    nodes = []
    for i in range(n):
        nodes.append(i)
    m = Model("OP2")
    
    m.setParam(GRB.Param.PoolSolutions,numPaths)
    m.setParam(GRB.Param.PoolSearchMode,2)
    
    inflow = np.zeros(n)
    inflow[s] = k
    inflow[n-1] = -k
    
    visited = m.addVars(nodes, vtype=GRB.BINARY,  name="visited")
    flows = m.addVars(nodes,nodes, vtype = GRB.INTEGER, name="flows")
    
    # Flow conservation
    m.addConstrs(
    (flows.sum('*',j) + inflow[j] == flows.sum(j,'*') for j in nodes), "Conservation")
    
    # Flow only on existing arcs
    m.addConstrs((flows[i,j] <= adjacency[i,j] for i in nodes for j in nodes),"EdgesPresent")
    
    # Visited nodes
    m.addConstrs((visited[j] <= flows.sum('*',j) for j in nodes), "WhetherVisited")
    
#     obj = quicksum(visited[j] * prizes[j])
    
    # Objective Function
    m.setObjective((quicksum(visited[j]*prizes[j] for j in nodes)), GRB.MAXIMIZE)
    
    m.optimize()
    if m.status == GRB.Status.OPTIMAL:
        print(m.status)
        solution = m.getAttr('x',flows)
        for i in nodes:
            for j in nodes:
                if solution[i,j] > 0:
                    print('%s -> %s: %g' % (i, j, solution[i,j]))
    
    #save model
    
    return m,flows

# Assumes k = 1, constructs path from solution
def construct_path(m,n,flows):
    solution = m.getAttr('xn',flows)
    path = []
    nodes = range(n)
    for i in nodes:
        for j in nodes:
            if solution[i,j] > 0:
                path.append(j)
    return path

def score_paths(paths,k,prizes):
    scores = np.zeros(k,dtype=int)
    prizes_copy = prizes.copy()
    print(scores)
    go = True
    step = -1
    while go:
        go = False
        step += 1
        for player in range(k):
            path = paths[player]
            if len(path) > step:
                scores[player] += prizes_copy[path[step]]
                prizes_copy[path[step]] = 0
                go = True
    return scores


In [50]:
def unreserved_best_response(adjacency,n,prizes,first_path):
    nodes = []
    for i in range(n):
        nodes.append(i)
    m = Model("OP2")
    
    inflow = np.zeros(n)
    inflow[0] = 1
    inflow[n-1] = -1
    
    visited = m.addVars(nodes, vtype=GRB.BINARY,  name="visited")
    flows = m.addVars(nodes,nodes, vtype = GRB.INTEGER, name="flows")
    arrivals2 = m.addVars(nodes, vtype=GRB.INTEGER, name="arrivals2")
#     arrivals1 = m.addVars(nodes, name="arrivals1")
    second_prizes = m.addVars(nodes, vtype = GRB.BINARY, name = "second_prizes")
    
    full_arrivals1 = np.zeros(n)+n
    full_arrivals1[0] = 0
    for i in range(len(first_path)):
        temp = first_path[i]
        full_arrivals1[temp] = i+1
    
        # Flow conservation
    m.addConstrs((flows.sum('*',j) + inflow[j] == flows.sum(j,'*') for j in nodes), "Conservation")
    
    # Flow only on existing arcs
    m.addConstrs((flows[i,j] <= adjacency[i,j] for i in nodes for j in nodes),"EdgesPresent")
    
    # Prizes uncollected if not visited
    m.addConstrs((second_prizes[j] <= flows.sum('*',j) for j in nodes), "WhetherVisited")
    
    # Prizes uncollected if not visited FIRST
    m.addConstrs((second_prizes[i] <= full_arrivals1[i] - arrivals2[i] for i in nodes), "visitedFirst")
    
    # Enforce correct arrival times
    #m.addConstrs((arrivals2[i] >= (arrivals2[j] + 1) * flows(j,i) for i >=1 in nodes for j < i in nodes), "arrivalEnforced")
    m.addConstrs((arrivals2[i] >= arrivals2[j] + (flows[j,i] - 1)* n + flows[j,i] for i in nodes for j in nodes), "arrivalEnforced")
#     m.addConstrs((arrivals2[i] >= arrivals2[j] -1 + 2*flows[j,i] for i in nodes for j in nodes), "arrivalEnforced")
    
    # Set starting point
    m.addConstr(arrivals2[0] == 0, "startEnforced")
    
    m.setObjective((quicksum(second_prizes[j]*prizes[j] for j in nodes)), GRB.MAXIMIZE)
    
    m.optimize()
    if m.status == GRB.Status.OPTIMAL:
        print(m.status)
        solution = m.getAttr('x',flows)
        for i in nodes:
            for j in nodes:
                if solution[i,j] > 0:
                    print('%s -> %s: %g' % (i, j, solution[i,j]))
    
    #save model
    
    return m,flows
    
# Analysis for k best paths (2 players)
# def k_paths(adj,n,prizes,num_best):
#     model, flows = solve_TOP


In [115]:


class State:
    def __init__(self,node1,node2,prizes_between):
        self.n1 = node1
        self.n2 = node2
        self.prizes = prizes_between
    
state_space = {}

def construct_turn_paths(state_space,initial_state,n):
    def recursive_path_constructor(state,n):
        nonlocal turn
        nonlocal path1
        nonlocal path2
        next_state = state_space[state]
        if state.n1 < n-1:
            if turn == 0:
                path1.append(next_state['next_step'])
            else:
                path2.append(next_state['next_step'])
        turn = (turn + 1) % 2
        if state.n2 < n-1:
            print(next_state)
            recursive_path_constructor(next_state['next_state'],n)
    
    turn = 0
    path1 = [0]
    path2 = [0]
    recursive_path_constructor(initial_state,n)
    return path1,path2

def turn_wrapper(adj,n,prizes):
    def turn_recurser(adj,n,prizes,state):
        nonlocal state_space
        next_step = -1
        value = 0
        n1 = state.n1
        n2 = state.n2
        if state in state_space:
            return state_space[state]
        else:
            # Check if next player is at the final node
            if n2 == n-1:
                # Check if current player is at the final node
                if n1 == n-1:
                    state_space[state] = {'val1':0,'val2':0,'next_state':state,'next_step': n-1}

                # Current player remains
                else:
                    new_prizes = prizes
                    n1 = state.n1
                    for i in range(n1+1,n):
                        new_prizes[i] = state.prizes[i-n1-1]
                    model,flows = solve_TOP(adj,n,1,new_prizes,1,n1)
                    state_space[state] = {'val1':model.objval,'val2':0, 'next_state':State(n-1,n-1,[]),'next_step': construct_path(model,n,flows)}

            # Check if current player is at final node
            elif n1 == n-1:
                new_state = State(state.n2,n-1,state.prizes)
                turn_recurser(adj,n,prizes,new_state)
                state_space[state] = {'val1':0, 'val2': state_space[new_state]['val1'],'next_state':new_state,'next_step':n-1}

            # If neither player is at final node
            else:
                row = adj[n1]
                next_moves = []
                next_step = n-1
                for i in range(n):
                    if row[i] == 1:
                        next_moves.append(i)

                # Assumes non-negative prizes
                state_space[state] = {'val1':-1,'val2':-1,'next_state':State(-1,-1,[]),'next_step':n-1}
                for new_step in next_moves:
                    step_prize = 0
                    new_prizes = []
                    if new_step == n2:
                        new_prizes = []
                    elif new_step > n2:
                        if n2 >= n1:
                            print([n1,n2])
                            new_prizes = prizes[n2+1:new_step+1]
                            step_prize = prizes[new_step]
                            print(new_prizes)
                            new_prizes[new_step-(n2+1)] = 0
                            print(new_prizes)
                        else: 
                            temp = prizes[n1+1:new_step+1]
                            step_prize = prizes[new_step]
                            temp[new_step-(n1+1)] = 0
                            new_prizes = state.prizes + temp
                    else:
                        new_prizes = state.prizes[new_step-(n1+1):]
                        step_prize = state.prizes[new_step-n1]
                    new_state = State(state.n2,new_step,new_prizes)
                    turn_recurser(adj,n,prizes,new_state)
                    if step_prize+state_space[new_state]['val2'] > state_space[state]['val1']:
                        state_space[state] = {'val1':step_prize+state_space[new_state]['val2'],
                                              'val2':state_space[new_state]['val1'], 'next_state':new_state,'next_step':new_step}
          
    state_space = {}
    state = State(0,0,[])
    turn_recurser(adj,n,prizes,state)
    return (state_space,state)



In [110]:
n = 5
p = 1
k = 1
prizes = []
for i in range(n):
    prizes.append(random.randint(1,1))
prizes[0] = 0
prizes[n-1] = 0
adj = rand_DAG(n,p)
print(adj)
print(prizes)
print(adj[0])

# print(state_space)
# wrapper([],3,[])
# print(state_space)

[[ 0.  1.  1.  1.  1.]
 [ 0.  0.  1.  1.  1.]
 [ 0.  0.  0.  1.  1.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.]]
[0, 1, 1, 1, 0]
[ 0.  1.  1.  1.  1.]


In [116]:
# model,flows = solve_TOP(adj,n,k,prizes,1,0);
# temp = model.getAttr('x');
# print(model.getAttr('x'));
# print(construct_path(model,n,flows))
# print(model.objval)

# holder,initial_state = turn_wrapper(adj,n,prizes)
print(holder)
print(construct_turn_paths(holder,initial_state,n))
# model.setParam(GRB.Param.SolutionNumber,99)
# print(model.getAttr('xn'))
# first_path = construct_path(model,n,flows);
# print(first_path)
# new_model,new_flows = unreserved_best_response(adj,n,prizes,first_path);
# second_path = construct_path(new_model,n,new_flows);
# print(second_path)
# print(score_paths([first_path,second_path],2,prizes))


{<__main__.State object at 0x00000000058878D0>: {'val1': 2.0, 'val2': 1, 'next_state': <__main__.State object at 0x0000000005887C88>, 'next_step': 1}, <__main__.State object at 0x0000000005887C88>: {'val1': 1, 'val2': 1.0, 'next_state': <__main__.State object at 0x00000000058879B0>, 'next_step': 1}, <__main__.State object at 0x00000000058879B0>: {'val1': 1.0, 'val2': 1, 'next_state': <__main__.State object at 0x00000000079B70F0>, 'next_step': 2}, <__main__.State object at 0x00000000079B70F0>: {'val1': 1, 'val2': 0.0, 'next_state': <__main__.State object at 0x0000000007D056D8>, 'next_step': 3}, <__main__.State object at 0x00000000079B7BE0>: {'val1': 1, 'val2': 0.0, 'next_state': <__main__.State object at 0x00000000079B7978>, 'next_step': 3}, <__main__.State object at 0x00000000079B7978>: {'val1': 0.0, 'val2': 0, 'next_state': <__main__.State object at 0x00000000079B7470>, 'next_step': 3}, <__main__.State object at 0x00000000079B7470>: {'val1': 0, 'val2': -0.0, 'next_state': <__main__.St

In [330]:

model,flows = solve_TOP(adj,n,1,prizes,100)
temp = model.getAttr('x')
print(model.getAttr('x'))
# print(construct_path(model,n,flows))
model.setParam(GRB.Param.SolutionNumber,99)
print(model.getAttr('xn'))
print(construct_path(model,n,flows))

Changed value of parameter PoolSolutions to 100
   Prev: 10  Min: 1  Max: 2000000000  Default: 10
Changed value of parameter PoolSearchMode to 2
   Prev: 0  Min: 0  Max: 2  Default: 0
Optimize a model with 120 rows, 110 columns and 390 nonzeros
Variable types: 0 continuous, 110 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1.0000000
Presolve removed 105 rows and 77 columns
Presolve time: 0.00s
Presolved: 15 rows, 33 columns, 71 nonzeros
Variable types: 0 continuous, 33 integer (33 binary)
Found heuristic solution: objective 7.0000000

Root relaxation: objective 8.000000e+00, 18 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       8.0000000    8.00000  0.00%     -

In [321]:

x = model.getAttr('x',flows)
y = model.getAttr('x')
print(x)
print(y)


{(0, 0): 0.0, (0, 1): 1.0, (0, 2): 0.0, (0, 3): 0.0, (0, 4): 0.0, (0, 5): -0.0, (0, 6): 0.0, (0, 7): -0.0, (0, 8): -0.0, (0, 9): -0.0, (1, 0): 0.0, (1, 1): 0.0, (1, 2): 1.0, (1, 3): -0.0, (1, 4): 0.0, (1, 5): -0.0, (1, 6): -0.0, (1, 7): -0.0, (1, 8): 0.0, (1, 9): -0.0, (2, 0): 0.0, (2, 1): 0.0, (2, 2): 0.0, (2, 3): 0.0, (2, 4): 0.0, (2, 5): 1.0, (2, 6): -0.0, (2, 7): 0.0, (2, 8): 0.0, (2, 9): 0.0, (3, 0): 0.0, (3, 1): 0.0, (3, 2): 0.0, (3, 3): 0.0, (3, 4): 0.0, (3, 5): 0.0, (3, 6): 0.0, (3, 7): 0.0, (3, 8): 0.0, (3, 9): 0.0, (4, 0): 0.0, (4, 1): 0.0, (4, 2): 0.0, (4, 3): 0.0, (4, 4): 0.0, (4, 5): -0.0, (4, 6): -0.0, (4, 7): 0.0, (4, 8): 0.0, (4, 9): 0.0, (5, 0): 0.0, (5, 1): 0.0, (5, 2): 0.0, (5, 3): 0.0, (5, 4): 0.0, (5, 5): 0.0, (5, 6): 1.0, (5, 7): 0.0, (5, 8): -0.0, (5, 9): 0.0, (6, 0): 0.0, (6, 1): 0.0, (6, 2): 0.0, (6, 3): 0.0, (6, 4): 0.0, (6, 5): 0.0, (6, 6): 0.0, (6, 7): 1.0, (6, 8): -0.0, (6, 9): -0.0, (7, 0): 0.0, (7, 1): 0.0, (7, 2): 0.0, (7, 3): 0.0, (7, 4): 0.0, (7, 5): 0